In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem1234561"}
# %mpy -s {"/dev/cu.usbmodem2101"}
%mpy -s {"/dev/cu.usbmodem101"}

In [ ]:
!mpremote reset

In [ ]:
# %%micropython
import time
from machine import lightsleep
from mp_libs.power import powerfeather

pf = powerfeather.PowerFeather(batt_type=powerfeather.BatteryType.GENERIC_3V7, batt_cap=1050)

print("Starting loop...")
while True:
    print("Toggling...")
    pf.led_toggle()
    time.sleep_ms(250)
    # lightsleep(1000)


In [ ]:
# %%micropython

from app import main

main()

In [ ]:
# %%micropython
import time

try:
    print("My test loop")
    time.sleep(5)
    raise Exception("My test exception")
except Exception as exc:
    import io
    import sys
    import time
    from config import config
    from mp_libs import logging
    from mp_libs.power.powerfeather import PowerFeather

    # TODO: Fix this
    # Close down any other handlers so they aren't used here
    logging.shutdown()

    # Create logger
    logger = logging.getLogger("main")
    logger.setLevel(config.get("logging_level", logging.INFO))
    file_handler = logging.FileHandler("Exception_Log.txt", "a")
    file_handler.setLevel(config.get("logging_level", logging.INFO))
    file_handler.setFormatter(logging.Formatter("%(mono)d %(levelname)s-%(name)s:%(message)s"))
    logger.addHandler(file_handler)

    # Log message to file
    logger.critical("Caught unexpected exception:")
    buf = io.StringIO()
    sys.print_exception(exc, buf)
    print(buf.getvalue())
    logger.critical(f"{buf.getvalue()}")
    logger.critical("Looping...")

    # Close/flush file handler
    logging.shutdown()

    # NOTE: A hard reset will cause RTC memory to be lost
    if config.get("debug", False):
        print("Blinking")
        pf = PowerFeather(first_boot=False, init_periphs=False)
        while True:
            pf.led_toggle()
            time.sleep(0.25)
    else:
        import machine
        machine.reset()